In [1]:
%load_ext sql
%sql duckdb:///:memory:
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Load CSV file into a table in memory

In [2]:
%sql CREATE TABLE popular_movies AS SELECT * FROM 'popular_10000_movies_tmdb.csv'

Count
10000


Get a quick look at some interesting columns

In [6]:
%%sql 
SELECT title, vote_average, popularity, budget, revenue, runtime
FROM popular_movies LIMIT 10

title,vote_average,popularity,budget,revenue,runtime
The Pope's Exorcist,7.4,5089.969,18000000,65675816,103
Ant-Man and the Wasp: Quantumania,6.6,4665.438,200000000,464566092,125
The Super Mario Bros. Movie,7.5,3935.55,100000000,1121048165,92
Ghosted,7.2,2791.532,0,0,120
Shazam! Fury of the Gods,6.8,2702.593,125000000,133437105,130
Avatar: The Way of Water,7.7,2280.912,460000000,2319331580,192
Guardians of the Galaxy Volume 3,8.3,2520.308,250000000,289312702,150
Scream VI,7.3,1862.283,35000000,168541093,123
Creed III,7.3,1894.044,75000000,269000000,116
Dungeons & Dragons: Honor Among Thieves,7.5,1655.052,151000000,202484920,134


Look at the top 10 most popular movies

In [32]:
%%sql

SELECT title, vote_average, vote_count, popularity, (revenue - budget) / 1000000 AS profit_in_millions
FROM popular_movies
WHERE vote_count > 10000
ORDER BY vote_average DESC
LIMIT 10

title,vote_average,vote_count,popularity,profit_in_millions
The Godfather,8.7,17865,113.216,239
The Shawshank Redemption,8.7,23715,84.139,3
Schindler's List,8.6,14028,57.91,299
The Godfather Part II,8.6,10802,55.942,89
Spirited Away,8.5,14246,74.783,255
The Dark Knight,8.5,29634,81.582,819
The Green Mile,8.5,15357,67.506,226
The Lord of the Rings: The Return of the King,8.5,21539,65.571,1024
Pulp Fiction,8.5,25104,59.938,206
Parasite,8.5,15673,70.397,246


Top 10 movies with bad ratings that turn a profit

In [42]:
%%sql

SELECT 
    title, 
    vote_average,
    (revenue - budget) / 1000000 AS profit_in_millions
FROM popular_movies
WHERE 
    revenue - budget > 0 AND
    vote_average < 6
ORDER BY profit_in_millions DESC
LIMIT 10

title,vote_average,profit_in_millions
Transformers: Age of Extinction,5.9,894
Full River Red,5.8,673
Batman v Superman: Dawn of Justice,5.9,623
2012,5.8,591
Suicide Squad,5.9,571
Fifty Shades of Grey,5.9,531
The Smurfs,5.8,453
Clash of the Titans,5.9,368
Alvin and the Chipmunks: The Squeakquel,5.6,368
Teenage Mutant Ninja Turtles,5.9,360


Genres that make the most money

In [44]:
%%sql

SELECT 
    genres,
    SUM(revenue - budget) AS total_profit
FROM popular_movies
GROUP BY genres
HAVING SUM(revenue - budget) > 0
ORDER BY total_profit DESC
LIMIT 10


genres,total_profit
"['Action', 'Adventure', 'Science Fiction']",15795068792
['Comedy'],11996953283
"['Adventure', 'Action', 'Science Fiction']",11762937055
"['Comedy', 'Romance']",10850859850
['Drama'],9367367836
"['Action', 'Adventure', 'Fantasy']",8580048698
"['Drama', 'Romance']",7738721098
"['Adventure', 'Fantasy', 'Action']",6264710538
"['Comedy', 'Drama', 'Romance']",5237230258
"['Adventure', 'Action', 'Thriller']",4969060289
